In [149]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import ergast_loader 

In [161]:
urls = [
    "https://press.pirelli.com/2019-german-grand-prix---race/",
    "https://press.pirelli.com/2020-belgian-grand-prix--race/",
    "https://press.pirelli.com/2020-british-grand-prix---race/"
] 

In [158]:
ergast = ergast_loader.ErgastLoader('data//ergast_data')

In [177]:
def generate_urls(year, race):
    urls = []
    url_blank = 'http://press.pirelli.com/{}-{}{}-race/'
    for i in range(5):
        urls.append(url_blank.format(str(year), race_str, '-' * i))
    return urls
generate_urls(2020, 'german-grand-prix')

['http://press.pirelli.com/2020-abu-dhabi-grand-prix-race/',
 'http://press.pirelli.com/2020-abu-dhabi-grand-prix--race/',
 'http://press.pirelli.com/2020-abu-dhabi-grand-prix---race/',
 'http://press.pirelli.com/2020-abu-dhabi-grand-prix----race/',
 'http://press.pirelli.com/2020-abu-dhabi-grand-prix-----race/']

In [182]:
# guess 2019 urls

races = ergast.data['races'].copy()
races_2019 = races.loc[races['year']== 2019, 'name'].to_list()

dfs = []
for race in races_2019:
    race_str = race.lower().replace(' ', '-')
    print(race_str)
    for url in generate_urls(2019, race_str):
        try:
            df = scrape_tire_usage_data(url)
            df['year'] = 2019
            df['race'] = race_str
            dfs.append(df)
            break
        except ValueError:
            continue
        
        print('failed to find race data:', race_str)    

australian-grand-prix
bahrain-grand-prix
chinese-grand-prix
azerbaijan-grand-prix
spanish-grand-prix
monaco-grand-prix
canadian-grand-prix
french-grand-prix
austrian-grand-prix
british-grand-prix
german-grand-prix
hungarian-grand-prix
belgian-grand-prix
italian-grand-prix
singapore-grand-prix
russian-grand-prix
japanese-grand-prix
mexican-grand-prix
united-states-grand-prix
brazilian-grand-prix
abu-dhabi-grand-prix


In [184]:
race_tyre_data_2019 = pd.concat(dfs)

In [185]:
race_tyre_data_2019

,driver,stop,value,year,race
0,BOT,0,C4u,2019,australian-grand-prix
1,HAM,0,C4u,2019,australian-grand-prix
2,VER,0,C4u,2019,australian-grand-prix
3,VET,0,C4u,2019,australian-grand-prix
4,LEC,0,C4u,2019,australian-grand-prix
...,...,...,...,...,...
44,VET,2,C4n (38),2019,abu-dhabi-grand-prix
49,SAI,2,C4n (41),2019,abu-dhabi-grand-prix
50,RIC,2,C5u (42),2019,abu-dhabi-grand-prix
55,GIO,2,C4n (26),2019,abu-dhabi-grand-prix


In [162]:
def scrape_tire_usage_data(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    
    df = pd.read_html(str(soup), match=r'(.*PIT 1.*)')[0]
    
    if isinstance(df.columns, pd.core.indexes.multi.MultiIndex):
        df.columns = df.columns.droplevel()
    
    df.columns = [c.lower() for c in df.columns]
    df.drop('car', axis=1, inplace=True)

    df = df.melt('driver').copy()
    
    df.rename({'variable': 'stop'}, axis=1, inplace=True)
    df['stop'] = df['stop'].str.replace('start', '0')
    df['stop'] = df['stop'].str.replace('pit ', '')
    df['stop'] = df['stop'].astype(int)
    
    df = df.dropna()
    
    return df

for url in urls:
    print(url)
    print(scrape_tire_usage_data(url))

https://press.pirelli.com/2019-german-grand-prix---race/
    driver  stop     value
0      VER     0        Wn
1      VET     0        Wn
2      KVY     0        Wn
3      STR     0        Wn
4      SAI     0        Wn
..     ...   ...       ...
110    HAM     5  C4u (53)
111    KUB     5  C4u (57)
112    RUS     5  C4u (57)
129    MAG     6  C4u (57)
130    HAM     6  C4u (56)

[98 rows x 3 columns]
https://press.pirelli.com/2020-belgian-grand-prix--race/
   driver  stop     value
0     HAM     0       C3u
1     BOT     0       C3u
2     VER     0       C3u
3     RIC     0       C4u
4     OCO     0       C4u
5     ALB     0       C4u
6     NOR     0       C4u
7     GAS     0       C2n
8     STR     0       C4u
9     PER     0       C4u
10    KVY     0       C3n
11    RAI     0       C3n
12    VET     0       C3n
13    LEC     0       C4n
14    GRO     0       C3n
15    LAT     0       C3n
16    MAG     0       C3n
17    GIO     0       C3n
18    RUS     0       C3n
19    SAI     0    

ValueError: No tables found